## Prepare setups

In [177]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys

chrome_driver = "C://Users//amged//developer//chromedriver.exe"

In [178]:
yolo_map_link = "https://home.yolo-japan.com/en/tokyo/list?priceTo=95&room=2,3&dateBuild=30&property_type_id=3102,3101&buildingEquipment=23201,20101&perPage=50&page=1"

driver =  webdriver.Chrome(executable_path=chrome_driver)
driver.get(yolo_map_link)
try:
    # Close the pop-up menu if it appears
    driver.find_element(by=By.CSS_SELECTOR, value='.modal-body ul li').click()
except:
    pass
# Change the view
driver.find_element(by=By.XPATH ,value='/html/body/div/div/div/div[2]/div[2]/section[3]/div/div[2]/div/div/div[1]/div[1]/div[3]/div[2]/button[2]').click()

C:\Users\amged\AppData\Local\Temp/ipykernel_18096/281325430.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver =  webdriver.Chrome(executable_path=chrome_driver)


In [179]:
# Get houses tags
house = driver.find_elements(by=By.CSS_SELECTOR, value='.property-wrapper')[0]
# house type and name
full_name = house.find_element(by=By.TAG_NAME, value='h5').text
# house Type
apartment_type = re.match('^[a-zA-Z0-9 ]*', full_name).group(0)[:-1]
# house name
apartment_name = full_name.replace(apartment_type+' ',"")
# hyperlink
apartment_link = house.find_element(by=By.TAG_NAME, value='a').get_attribute('href')
# Apartment id
ID = re.search('[0-9]*$',apartment_link)

In [97]:
# Get age an address
address, age = house.find_elements(by=By.CSS_SELECTOR, value='.txt-info span')
address = address.text
age = re.search('[0-9]+' , age.text.split('/')[1]).group()

In [206]:
appartment = house.find_elements(by=By.CSS_SELECTOR, value='.property-list .item-right')[0]
monthly_payment = appartment.find_element(by=By.CSS_SELECTOR, value='.price').text
monthly_payment = sum([int(re.sub("[^0-9.]",'',i)) for i in monthly_payment.split('/')])
appartment.find_element(by=By.CSS_SELECTOR, value='.w-40').text
monthly_payment

69000

In [186]:
appartment = house.find_elements(by=By.CSS_SELECTOR, value='.property-list .item-right')[0]

appartment_link = appartment.find_element(by=By.CSS_SELECTOR, value='a').get_attribute('href')
appartment_id = re.search('[0-9]*$',appartment_link).group(0)
# get appartment details as a list
appartment = appartment.text.split('\n')
montly_payment = sum([int(re.sub("[^0-9.]",'',i)) for i in appartment[0].split('/')])
floor = int(re.sub('[^0-9]','',appartment[1]))
room_type = appartment[2]
area = float(appartment[3].replace('m2',""))
# One Tiome Payment
otp = sum([int(re.sub("[^0-9.]",'',i)) for i in appartment[-1].split('/')])

ValueError: invalid literal for int() with base 10: '40002126.082'

In [170]:
class House:
    def __init__(self, house):
        # Get houses tags
        self.house = house
        # house type and name
        self.full_name = self.house.find_element(
            by=By.TAG_NAME, value='h5').text
        # house Type
        self.appartment_type = re.match(
            '^[a-zA-Z0-9 ]*', full_name).group(0)[:-1]
        # house name
        self.appartment_name = full_name.replace(self.appartment_type+' ', "")
        # Get address and building age
        self.address, self.age = self.house.find_elements(
            by=By.CSS_SELECTOR, value='.txt-info span')
        self.address = self.address.text
        self.age = int(
            re.search('[0-9]+', self.age.text.split('/')[1]).group())
        # get appartments
        self.appartments = house.find_elements(
            by=By.CSS_SELECTOR, value='.property-list .item-right')


class Appartment:
    def __init__(self, appartment):
        self.appartment = appartment
        self.appartment_link = self.appartment.find_element(
            by=By.CSS_SELECTOR, value='a').get_attribute('href')
        self.appartment_id = re.search(
            '[0-9]*$', self.appartment_link).group(0)

        # get appartment details as a list
        self.appartment = self.appartment.text.split('\n')
        self.montly_payment = sum([int(re.sub("[^0-9.]", '', i))
                                  for i in self.appartment[0].split('/')])
        self.floor = int(re.sub('[^0-9]', '', self.appartment[1]))
        self.room_type = self.appartment[2]
        self.area = float(self.appartment[3].replace('m2', ""))
        # One Tiome Payment
        self.otp = sum([int(re.sub("[^0-9.]", '', i))
                        for i in self.appartment[-1].split('/')])


In [171]:
columns = ['ID', 'Name', 'Type','Built In', 'Address', 'Floor Number',
           'Room Type', 'Area', 'URL', 'Monthly Payment']
df = pd.DataFrame(columns=columns)

In [172]:
houses = driver.find_elements(by=By.CSS_SELECTOR, value='.property-wrapper')

for house in houses:
    house = House(house)
    for appartment in house.appartments:
        appartment = Appartment(appartment)
        if appartment.appartment_id not in df['ID']:
            data = {'ID':appartment.appartment_id,
                    'Name':house.appartment_name,
                    'Type':house.appartment_type,
                    'Built In':2022-house.age,
                    'Address':house.address,
                    'Floor Number':appartment.floor_number,
                'Room Type':appartment.room_type,
                'Area':appartment.area,
                'URL':appartment.appartment_link,
                'Monthly Payment':appartment.monthly_payment}
            df = df.append(data, ignore_index = True)
    break

df

ValueError: invalid literal for int() with base 10: '40002126.082'

In [176]:
house.appartments[0]

<selenium.webdriver.remote.webelement.WebElement (session="f84491a357b94e22d2311a528766fc42", element="66f45266-5489-41ab-9acd-d4a1b5a76781")>